### Notebook Setup

In [169]:
#Import libraries

import pandas as pd
import json
import requests # library to handle requests
from requests import get
import re # Regular expression
import ast
import matplotlib.pyplot as plt

from itertools import islice
from geopy.geocoders import Nominatim

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from bs4 import BeautifulSoup

In [72]:
pd.set_option('display.max_colwidth', None, 'display.max_rows', None)

In [78]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

___

### 1. Scrape data from online

In [138]:
#1.#https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E972&maxBedrooms=5&maxPrice=800000&minPrice=350000&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=newHome%2CsharedOwnership%2Cretirement&furnishTypes=&keywords=

#2#https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E94441&maxPrice=1000000&minPrice=300000&index=24&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=retirement&furnishTypes=&keywords=

#KT area: https://en.wikipedia.org/wiki/KT_postcode_area

all_data = pd.DataFrame([])

base_url= ("https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E94441&maxPrice=1000000&minPrice=300000&index=24&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=retirement&furnishTypes=&keywords=")

results = requests.get(base_url).text
soup = BeautifulSoup(results, "html.parser")

find = soup.find_all("script")

reg = r"window.jsonModel\s+=\s+(\{.*?\</script>)"

pattern = re.compile(reg)
match = pattern.findall(str(find))

match_strip = match[0].rstrip("</script>")

s1 = json.dumps(match_strip)
json_data = json.loads(s1) 

s3 = json.loads(str(json_data))

pagination =  s3.get("pagination").get("options")

pagination = pd.DataFrame(pagination)



for value in islice(pagination.iloc[:,0],0,42):
    print(value)

    iter_url = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E94441&maxPrice=1000000&minPrice=300000&index={}&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&secondaryDisplayPropertyType=housesandflats&includeSSTC=false&mustHave=&dontShow=retirement&furnishTypes=&keywords=".format(value)
    
   
   
    results = requests.get(iter_url).text
    soup = BeautifulSoup(results, "html.parser")
    
    find = soup.find_all("script")
    
    reg = r"window.jsonModel\s+=\s+(\{.*?\</script>)"

    pattern = re.compile(reg)
    match = pattern.findall(str(find))

    match_strip = match[0].rstrip("</script>")

    s1 = json.dumps(match_strip)
    json_data = json.loads(s1) 

    s3 = json.loads(str(json_data))
    
    list =  s3.get("properties")
    
    df = pd.DataFrame(list)
    df.drop(["propertyImages"], axis=1, inplace=True)

    all_data = all_data.append(df, ignore_index=True)
    
    
    
    
pd.set_option("display.max_rows", df.shape[0]+1)


all_data.shape
all_data

0
24
48
72
96
120
144
168
192
216
240
264
288
312
336
360
384
408
432
456
480
504
528
552
576
600
624
648
672
696
720
744
768
792
816
840
864
888
912
936
960
984


(1050, 47)

,id,bedrooms,bathrooms,numberOfImages,numberOfFloorplans,numberOfVirtualTours,summary,displayAddress,countryCode,location,...,onlineViewingsAvailable,heading,propertyTypeFullDescription,enhancedListing,displayStatus,formattedBranchName,addedOrReduced,isRecent,formattedDistance,hasBrandPlus
0,95544089,2,2.0,13,2,0,** GATED DEVELOPMENT/2 BATHROOMS/LOFT SPACE/SET IN LARGE GROUNDS** This stunning top floor apartment has been modernised throughout and is positioned within a secure gated development within easy reach of Esher High Street and under a mile from Esher Mainline Train Station offering fast transport...,"Sherriff Close, Esher",GB,"{'latitude': 51.380854, 'longitude': -0.368775}",...,False,Featured Property,2 bedroom apartment for sale,False,,"by Bourne Estate Agents incorporating James Fancy, Esher",Reduced on 01/02/2021,False,,True
1,101081657,4,3.0,13,1,1,"Video Tour Available Located behind gates on a private road is this delightful and well-presented four bedroom, three bathroom townhouse. Set over three floors with study, kitchen, dining room and kitchen on the ground floor. The living room has a further balcony. Integral garage and c...","High Beeches, Weybridge, Surrey, KT13",GB,"{'latitude': 51.364078, 'longitude': -0.433442}",...,False,,4 bedroom town house for sale,False,,"by Jackson-Stops, Weybridge",Added on 14/12/2020,False,,True
2,77309661,4,NaN,2,0,1,"*LAUNCHING FEBRUARY 2021* A stylish collection of 1 & 2 bedroom apartments and 3, 4 & 5 bedroom houses in a magnificent tree-lined setting. Located in the sought-after Ashley Park neighbourhood. This is where you can sit back and feel truly at home.",Stompond Lane\r\nWalton-On-Thames\r\nKT12 1HB,GB,"{'latitude': 51.38009, 'longitude': -0.41621}",...,False,,4 bedroom semi-detached house for sale,False,,by London Square,Added on 20/01/2021,False,,True
3,73255458,2,2.0,11,1,1,"A spectacular air conditioned two bedroom apartment located within an exclusive gated development in Esher with onsite concierge. Located on the second floor, the property briefly comprises a large hallway and open plan living/dining room, separate fitted kitchen, a principal suite co...","Broomhill Court, Esher Close, Esher, KT10",GB,"{'latitude': 51.365766, 'longitude': -0.367829}",...,False,,2 bedroom apartment for sale,False,,"by Fine & Country, Richmond",Added on 15/09/2020,False,,True
4,76636641,6,0.0,26,1,1,"RARELY AVAILABLE DETACHED FAMILY HOME IN CUL-DE-SAC LOCATION CLOSE TO SCHOOLS AND STATION!!! Cairds are proud to bring to market for only the second time since new, this wonderful detached family home. Well loved and looked after for over 30 years, this substantial detached house is loca...","Boleyn Avenue, Epsom",GB,"{'latitude': 51.34777, 'longitude': -0.242032}",...,True,,6 bedroom detached house for sale,False,,"by Cairds The Estate Agents, Epsom - SALES",Reduced yesterday,True,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,103318841,5,2.0,12,1,0,Barnard Marcus are delighted and excited to welcome to the market this extended five bedroom semi-detached family home set on an extremely popular residential cul de sac. The property is situated within close proximity of Ewell Village and both Ewell West and East stations.,"Southville Close, EPSOM",GB,"{'latitude': 51.35307, 'longitude': -0.26579}",...,False,,5 bedroom semi-detached house for sale,False,,"by Barnard Marcus, Ewell",Added on 24/02/2021,False,,True
1046,66192291,2,2.0,15,1,0,Fabulous two bedroom first floor apartment situated in a highly regarded and sought after development of luxury homes in the superbly located Queens Reach development. The property has gated access and has the benefit of lovely communal landscaped gardens leading down to the River Mole. ...,"Queens Reach, East Molesey",GB,"{'latitude': 51.401489, 'longitude': -0.344583}",...,False,,2 bedroom apartment for sale,False,,"by Domains Property Services, East Molesey Sales",Reduced on 11/11/2020,False,,T

In [139]:
s3.get("properties")

[{'id': 103411883,
  'bedrooms': 3,
  'bathrooms': 0,
  'numberOfImages': 14,
  'numberOfFloorplans': 1,
  'numberOfVirtualTours': 0,
  'summary': "***NEW INSTRUCTION*** VIEWINGS FROM SATURDAY 6TH MARCH - A fine example of a superb three bedroom 'Berg' halls adjoining semi-detached family house located in an ideal position within easy walking distance of Ashtead's mainline station, local village shops and excellent schools close to hand whic...",
  'displayAddress': 'ASHTEAD, KT21',
  'countryCode': 'GB',
  'location': {'latitude': 51.32035, 'longitude': -0.300994},
  'propertySubType': 'Semi-Detached',
  'listingUpdate': {'listingUpdateReason': 'new',
   'listingUpdateDate': '2021-02-27T15:58:01Z'},
  'premiumListing': False,
  'featuredProperty': True,
  'price': {'amount': 675000,
   'frequency': 'not specified',
   'currencyCode': 'GBP',
   'displayPrices': [{'displayPrice': '£675,000',
     'displayPriceQualifier': ''}]},
  'customer': {'branchId': 11128,
   'brandPlusLogoURI': '/

In [140]:
all_data

,id,bedrooms,bathrooms,numberOfImages,numberOfFloorplans,numberOfVirtualTours,summary,displayAddress,countryCode,location,...,onlineViewingsAvailable,heading,propertyTypeFullDescription,enhancedListing,displayStatus,formattedBranchName,addedOrReduced,isRecent,formattedDistance,hasBrandPlus
0,95544089,2,2.0,13,2,0,** GATED DEVELOPMENT/2 BATHROOMS/LOFT SPACE/SET IN LARGE GROUNDS** This stunning top floor apartment has been modernised throughout and is positioned within a secure gated development within easy reach of Esher High Street and under a mile from Esher Mainline Train Station offering fast transport...,"Sherriff Close, Esher",GB,"{'latitude': 51.380854, 'longitude': -0.368775}",...,False,Featured Property,2 bedroom apartment for sale,False,,"by Bourne Estate Agents incorporating James Fancy, Esher",Reduced on 01/02/2021,False,,True
1,101081657,4,3.0,13,1,1,"Video Tour Available Located behind gates on a private road is this delightful and well-presented four bedroom, three bathroom townhouse. Set over three floors with study, kitchen, dining room and kitchen on the ground floor. The living room has a further balcony. Integral garage and c...","High Beeches, Weybridge, Surrey, KT13",GB,"{'latitude': 51.364078, 'longitude': -0.433442}",...,False,,4 bedroom town house for sale,False,,"by Jackson-Stops, Weybridge",Added on 14/12/2020,False,,True
2,77309661,4,NaN,2,0,1,"*LAUNCHING FEBRUARY 2021* A stylish collection of 1 & 2 bedroom apartments and 3, 4 & 5 bedroom houses in a magnificent tree-lined setting. Located in the sought-after Ashley Park neighbourhood. This is where you can sit back and feel truly at home.",Stompond Lane\r\nWalton-On-Thames\r\nKT12 1HB,GB,"{'latitude': 51.38009, 'longitude': -0.41621}",...,False,,4 bedroom semi-detached house for sale,False,,by London Square,Added on 20/01/2021,False,,True
3,73255458,2,2.0,11,1,1,"A spectacular air conditioned two bedroom apartment located within an exclusive gated development in Esher with onsite concierge. Located on the second floor, the property briefly comprises a large hallway and open plan living/dining room, separate fitted kitchen, a principal suite co...","Broomhill Court, Esher Close, Esher, KT10",GB,"{'latitude': 51.365766, 'longitude': -0.367829}",...,False,,2 bedroom apartment for sale,False,,"by Fine & Country, Richmond",Added on 15/09/2020,False,,True
4,76636641,6,0.0,26,1,1,"RARELY AVAILABLE DETACHED FAMILY HOME IN CUL-DE-SAC LOCATION CLOSE TO SCHOOLS AND STATION!!! Cairds are proud to bring to market for only the second time since new, this wonderful detached family home. Well loved and looked after for over 30 years, this substantial detached house is loca...","Boleyn Avenue, Epsom",GB,"{'latitude': 51.34777, 'longitude': -0.242032}",...,True,,6 bedroom detached house for sale,False,,"by Cairds The Estate Agents, Epsom - SALES",Reduced yesterday,True,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,103318841,5,2.0,12,1,0,Barnard Marcus are delighted and excited to welcome to the market this extended five bedroom semi-detached family home set on an extremely popular residential cul de sac. The property is situated within close proximity of Ewell Village and both Ewell West and East stations.,"Southville Close, EPSOM",GB,"{'latitude': 51.35307, 'longitude': -0.26579}",...,False,,5 bedroom semi-detached house for sale,False,,"by Barnard Marcus, Ewell",Added on 24/02/2021,False,,True
1046,66192291,2,2.0,15,1,0,Fabulous two bedroom first floor apartment situated in a highly regarded and sought after development of luxury homes in the superbly located Queens Reach development. The property has gated access and has the benefit of lovely communal landscaped gardens leading down to the River Mole. ...,"Queens Reach, East Molesey",GB,"{'latitude': 51.401489, 'longitude': -0.344583}",...,False,,2 bedroom apartment for sale,False,,"by Domains Property Services, East Molesey Sales",Reduced on 11/11/2020,False,,T

___

### 2. Pre-process Data

In [150]:
raw_data = all_data
raw_data.columns.tolist()

['id',
 'bedrooms',
 'bathrooms',
 'numberOfImages',
 'numberOfFloorplans',
 'numberOfVirtualTours',
 'summary',
 'displayAddress',
 'countryCode',
 'location',
 'propertySubType',
 'listingUpdate',
 'premiumListing',
 'featuredProperty',
 'price',
 'customer',
 'distance',
 'transactionType',
 'productLabel',
 'commercial',
 'development',
 'residential',
 'students',
 'auction',
 'feesApply',
 'feesApplyText',
 'displaySize',
 'showOnMap',
 'propertyUrl',
 'contactUrl',
 'staticMapUrl',
 'channel',
 'firstVisibleDate',
 'keywords',
 'keywordMatchType',
 'saved',
 'hidden',
 'onlineViewingsAvailable',
 'heading',
 'propertyTypeFullDescription',
 'enhancedListing',
 'displayStatus',
 'formattedBranchName',
 'addedOrReduced',
 'isRecent',
 'formattedDistance',
 'hasBrandPlus']

**Remove duplicated property listings by IDs**

In [151]:
raw_data.duplicated(subset=['id']).value_counts()
"""
Duplicate IDs exist when the listing is also a featured listing at the top of any page. IDs should be unique.
"""

False    1024
dtype: int64

'\nDuplicate IDs exist when the listing is also a featured listing at the top of any page. IDs should be unique.\n'

In [152]:
#Drop duplicates
raw_data.drop_duplicates(subset="id", inplace=True)
#Check for duplicates
raw_data.duplicated(subset=['id']).value_counts()

False    1024
dtype: int64

**Grab only required columns**

In [153]:
#Rather than dropping, choose which columns to keep
df_subset = raw_data.loc[:, ["id", "bedrooms", "bathrooms", "location", "propertySubType", "price", "auction", "channel", "productLabel" ]]
print(df_subset.shape)
df_subset

(1024, 9)


,id,bedrooms,bathrooms,location,propertySubType,price,auction,channel,productLabel
0,95544089,2,2.0,"{'latitude': 51.380854, 'longitude': -0.368775}",Apartment,"{'amount': 300000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£300,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",False,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1,101081657,4,3.0,"{'latitude': 51.364078, 'longitude': -0.433442}",Town House,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
2,77309661,4,NaN,"{'latitude': 51.38009, 'longitude': -0.41621}",Semi-Detached,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': 'Coming Soon', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': 'Coming Soon', 'spotlightLabel': True}"
3,73255458,2,2.0,"{'latitude': 51.365766, 'longitude': -0.367829}",Apartment,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': 'Premium Listing', 'spotlightLabel': False}"
4,76636641,6,0.0,"{'latitude': 51.34777, 'longitude': -0.242032}",Detached,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
...,...,...,...,...,...,...,...,...,...
1045,103318841,5,2.0,"{'latitude': 51.35307, 'longitude': -0.26579}",Semi-Detached,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': 'Virtual Tour', 'spotlightLabel': False}"
1046,66192291,2,2.0,"{'latitude': 51.401489, 'longitude': -0.344583}",Apartment,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
1047,94189550,2,2.0,"{'latitude': 51.38859, 'longitude': -0.29749}",Penthouse,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1048,75312948,2,1.0,"{'latitude': 51.392819, 'longitude': -0.31207}",Flat,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"


**Drop Auction properties**

In [159]:
auction_drop = df_subset.loc[df_subset["auction"] !=False]
auction_drop

print("number of auction listings dropped: ", auction_drop.shape[0])

df_subset = df_subset.drop(df_subset.index[auction_drop.index])
df_subset.shape


#Alt
# df_subset = df_subset[df_subset["auction"] == False]
# print("Deleted rows: " , df_subset[df_subset["auction"] != False].shape[0])
# print(df_subset.shape)

,id,bedrooms,bathrooms,location,propertySubType,price,auction,channel,productLabel
669,102525533,4,2.0,"{'latitude': 51.35049, 'longitude': -0.255419}",Detached,"{'amount': 650000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£650,000', 'displayPriceQualifier': 'Guide Price'}]}",True,BUY,"{'productLabelText': 'New Listing', 'spotlightLabel': False}"


number of auction listings dropped:  1


(1023, 9)

**Drop any non-Buy properties**

In [161]:
print("Delete rows: " , df_subset[df_subset["channel"] != "BUY"].shape[0])
df_subset = df_subset[df_subset["channel"] == "BUY"]
print(df_subset.shape)

Delete rows:  0
(1023, 9)


**Search and drop Null values**

For now drop rows where the bathroom value is NaN.    
For next time: See if you can best approximate the number of bathrooms based on property type and number of bedrooms and maybe also price of property.

In [165]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 0 to 1049
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1023 non-null   int64  
 1   bedrooms         1023 non-null   int64  
 2   bathrooms        942 non-null    float64
 3   location         1023 non-null   object 
 4   propertySubType  1023 non-null   object 
 5   price            1023 non-null   object 
 6   auction          1023 non-null   bool   
 7   channel          1023 non-null   object 
 8   productLabel     1023 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 72.9+ KB


In [167]:
df_subset["bathrooms"].value_counts(dropna = False)

2.0    512
1.0    261
3.0    125
NaN     81
0.0     32
4.0     11
5.0      1
Name: bathrooms, dtype: int64

In [168]:
df_subset["bathrooms"].value_counts(dropna = False, normalize = True)

2.0    0.500489
1.0    0.255132
3.0    0.122190
NaN    0.079179
0.0    0.031281
4.0    0.010753
5.0    0.000978
Name: bathrooms, dtype: float64

In [174]:
df_subset.dropna(subset=["bathrooms"], inplace=True)
df_subset.shape

(942, 9)

In [175]:
df_subset

,id,bedrooms,bathrooms,location,propertySubType,price,auction,channel,productLabel
0,95544089,2,2.0,"{'latitude': 51.380854, 'longitude': -0.368775}",Apartment,"{'amount': 300000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£300,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",False,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1,101081657,4,3.0,"{'latitude': 51.364078, 'longitude': -0.433442}",Town House,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
3,73255458,2,2.0,"{'latitude': 51.365766, 'longitude': -0.367829}",Apartment,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': 'Premium Listing', 'spotlightLabel': False}"
4,76636641,6,0.0,"{'latitude': 51.34777, 'longitude': -0.242032}",Detached,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
6,78349602,2,2.0,"{'latitude': 51.365766, 'longitude': -0.367829}",Apartment,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
...,...,...,...,...,...,...,...,...,...
1045,103318841,5,2.0,"{'latitude': 51.35307, 'longitude': -0.26579}",Semi-Detached,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': 'Virtual Tour', 'spotlightLabel': False}"
1046,66192291,2,2.0,"{'latitude': 51.401489, 'longitude': -0.344583}",Apartment,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': 'Guide Price'}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
1047,94189550,2,2.0,"{'latitude': 51.38859, 'longitude': -0.29749}",Penthouse,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1048,75312948,2,1.0,"{'latitude': 51.392819, 'longitude': -0.31207}",Flat,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",False,BUY,"{'productLabelText': '', 'spotlightLabel': False}"


**Drop auction and channel columns as no longer required**  

For practice, convert auction column to numerical binary}

In [176]:
df_subset['auction'] = (df['auction'] == 'TRUE').astype(int)
df_subset

,id,bedrooms,bathrooms,location,propertySubType,price,auction,channel,productLabel
0,95544089,2,2.0,"{'latitude': 51.380854, 'longitude': -0.368775}",Apartment,"{'amount': 300000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£300,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",0.0,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1,101081657,4,3.0,"{'latitude': 51.364078, 'longitude': -0.433442}",Town House,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",0.0,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
3,73255458,2,2.0,"{'latitude': 51.365766, 'longitude': -0.367829}",Apartment,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",0.0,BUY,"{'productLabelText': 'Premium Listing', 'spotlightLabel': False}"
4,76636641,6,0.0,"{'latitude': 51.34777, 'longitude': -0.242032}",Detached,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Offers in Excess of'}]}",0.0,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
6,78349602,2,2.0,"{'latitude': 51.365766, 'longitude': -0.367829}",Apartment,"{'amount': 1000000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£1,000,000', 'displayPriceQualifier': 'Guide Price'}]}",0.0,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
...,...,...,...,...,...,...,...,...,...
1045,103318841,5,2.0,"{'latitude': 51.35307, 'longitude': -0.26579}",Semi-Detached,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",NaN,BUY,"{'productLabelText': 'Virtual Tour', 'spotlightLabel': False}"
1046,66192291,2,2.0,"{'latitude': 51.401489, 'longitude': -0.344583}",Apartment,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': 'Guide Price'}]}",NaN,BUY,"{'productLabelText': '', 'spotlightLabel': False}"
1047,94189550,2,2.0,"{'latitude': 51.38859, 'longitude': -0.29749}",Penthouse,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",NaN,BUY,"{'productLabelText': 'Close to Station', 'spotlightLabel': False}"
1048,75312948,2,1.0,"{'latitude': 51.392819, 'longitude': -0.31207}",Flat,"{'amount': 550000, 'frequency': 'not specified', 'currencyCode': 'GBP', 'displayPrices': [{'displayPrice': '£550,000', 'displayPriceQualifier': ''}]}",NaN,BUY,"{'productLabelText': '', 'spotlightLabel': False}"


**Using data in Location - Create a Lat and Long column**